In [1]:
import networkx as nx
import matplotlib.pyplot as plt
import gurobipy as gp
from gurobipy import GRB


In [2]:
G = nx.read_gml("testgraph.gml")

vertices = G.nodes()
print("Vertices - ", vertices)

inflow = dict(G.nodes().data("weight"))
print("Inflow - ", inflow)

edges = G.edges()
print("Edges - ", edges)

capacity={}
cost={}
for i,j in edges:
    d = G.edges[i,j]
    capacity[i,j] = d['capacity']
    cost[i,j] = d['length']

print("Capacity - ", capacity)
print("Cost - ", cost)

T=8

# create model
m = gp.Model("Maximum Flow Problem")


# define variables
flow=gp.tupledict({})

for i, j in edges:
    flow.update(m.addVars(i, j, range(cost[i,j],T+1), ub=capacity[i,j], name = "flow"))
    

holdover_var = m.addVars(vertices, range(1,T+1), name = "holdover_var")


dummy=gp.tupledict({})
for i in vertices:
    dummy.update(m.addVars(i, lb=0, ub=inflow[i], name = "dummy"))

    
# flow conservation 
for i in vertices:
    for t in range(1,T):
        exp = holdover_var[i, t] + flow.sum('*', i, t) - holdover_var[i, t+1]
        for a,b in edges:
            if a==i:
                exp -= flow.sum(a, b, t + cost[a , b])
        m.addConstr(exp==0,"node")
    
            
# special case when t=0
for i in vertices:
    expr = dummy[i] - holdover_var[i, 1]
    for x,y in edges:
        if x==i:
            expr += flow[x, y, cost[x,y]]
    m.addConstr(expr==0)

#special case when t=T
m.addConstrs(flow.sum('*', j , T) + holdover_var[j, T] == 0 for j in vertices if j != 'O')

# objective maximize
obj=dummy.sum()
m.setObjective(obj, GRB.MAXIMIZE)

m.optimize()

for v in m.getVars():
    print(v.VarName, v.X)
    
print('Obj: ',m.ObjVal)

Vertices -  ['A', 'B', 'O']
Inflow -  {'A': 5, 'B': 10, 'O': 0}
Edges -  [('A', 'B'), ('A', 'O'), ('B', 'O'), ('B', 'A')]
Capacity -  {('A', 'B'): 2, ('A', 'O'): 4, ('B', 'O'): 1, ('B', 'A'): 1}
Cost -  {('A', 'B'): 1, ('A', 'O'): 3, ('B', 'O'): 2, ('B', 'A'): 1}
Set parameter Username
Academic license - for non-commercial use only - expires 2023-06-15
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (mac64[x86])
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 26 rows, 56 columns and 106 nonzeros
Model fingerprint: 0xe13c6341
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+01]
  RHS range        [0e+00, 0e+00]
Presolve removed 15 rows and 25 columns
Presolve time: 0.01s
Presolved: 11 rows, 31 columns, 50 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.5000000e+01   5.000000e+00   0.000000e+00      0s
      11    1.5000000e+01 

In [8]:
G = nx.read_gml("basicgraph.gml")

vertices = G.nodes()
print("Vertices - ", vertices)

inflow = dict(G.nodes().data("weight"))
print("Inflow - ", inflow)

edges = G.edges()
print("Edges - ", edges)

capacity={}
cost={}
for i,j in edges:
    d = G.edges[i,j]
    capacity[i,j] = d['capacity']
    cost[i,j] = d['length']

print("Capacity - ", capacity)
print("Cost - ", cost)

T=12

# create model
m = gp.Model("Maximum Flow Problem")


# define variables
flow=gp.tupledict({})

for i, j in edges:
    flow.update(m.addVars(i, j, range(cost[i,j],T+1), ub=capacity[i,j], name = "flow"))
    

holdover_var = m.addVars(vertices, range(1,T+1), name = "holdover_var")


dummy=gp.tupledict({})
for i in vertices:
    dummy.update(m.addVars(i, lb=0, ub=inflow[i], name = "dummy"))

    
# flow conservation 
for i in vertices:
    for t in range(1,T):
        exp = holdover_var[i, t] + flow.sum('*', i, t) - holdover_var[i, t+1]
        for a,b in edges:
            if a==i:
                exp -= flow.sum(a, b, t + cost[a , b])
        m.addConstr(exp==0,"node")
    
            
# special case when t=0
for i in vertices:
    expr = dummy[i] - holdover_var[i, 1]
    for x,y in edges:
        if x==i:
            expr += flow[x, y, cost[x,y]]
    m.addConstr(expr==0)

#special case when t=T
m.addConstrs(flow.sum('*', j , T) + holdover_var[j, T] == 0 for j in vertices if j != 'T')

# objective maximize
obj=dummy.sum()
m.setObjective(obj, GRB.MAXIMIZE)

m.optimize()

for v in m.getVars():
    print(v.VarName, v.X)
    
print('Obj: ',m.ObjVal)

Vertices -  ['A', 'B', 'C', 'T']
Inflow -  {'A': 5, 'B': 2, 'C': 4, 'T': 0}
Edges -  [('A', 'B'), ('A', 'C'), ('B', 'C'), ('B', 'T'), ('C', 'T')]
Capacity -  {('A', 'B'): 1, ('A', 'C'): 2, ('B', 'C'): 4, ('B', 'T'): 2, ('C', 'T'): 1}
Cost -  {('A', 'B'): 3, ('A', 'C'): 3, ('B', 'C'): 1, ('B', 'T'): 6, ('C', 'T'): 5}
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (mac64[x86])
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 51 rows, 99 columns and 191 nonzeros
Model fingerprint: 0xc256e291
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 5e+00]
  RHS range        [0e+00, 0e+00]
Presolve removed 38 rows and 59 columns
Presolve time: 0.01s
Presolved: 13 rows, 40 columns, 66 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.1000000e+01   6.500000e+00   0.000000e+00      0s
       8    1.1000000e+01   0.000000e+00   0.000000e+00      0s